<a href="https://colab.research.google.com/github/Caramel96/Samuel_BuckbyCE888/blob/main/lab_4%20/%20my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np 
import sklearn

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv', header=None
)

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.0,99.00,99.0,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.0,2.77,8.3,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.0,99.00,99.0,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.0,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.0,99.00,99.0,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [ ]:
df = df.drop([0], axis=1)

In [ ]:
arr = df

In [ ]:
rated = arr.replace(99, np.nan)

In [ ]:
def replace(orig, percentage=0.1):
  new_data = orig.copy()
  rated = np.where(~np.isnan(orig))
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [ ]:
new_arr, idx = replace(rated.values, 0.1)

In [ ]:
rated.values[idx[0][0], idx[1][0]]

-7.72

In [ ]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [ ]:
n_latent_factors = 2
user_ratings = new_arr
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1],n_latent_factors))

In [ ]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=10):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if not np.isnan(rating) and rating!=99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print('Iteration %d/%d:\tMSE=%.6f' % (iteration, iterations, mse))
            mse_history.append(mse)
    return mse_history

In [ ]:
num_iter = 10
hist = sgd(num_iter)  # Note how the MSE decreases with the number of iterations
plt.figure()
plt.plot(np.arange(0, num_iter, 10), hist)
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.show()

In [ ]:
predictions= latent_user_preferences.dot(latent_item_features.T)
predictions

array([[0.64093074, 1.21662622, 1.25606046, ..., 1.14315126, 1.3368977 ,
        0.44039606],
       [0.18876859, 0.32254424, 0.46606328, ..., 0.40362095, 0.49064301,
        0.32133802],
       [0.11394986, 0.2122437 , 0.2342147 , ..., 0.21083041, 0.24867408,
        0.10003118],
       ...,
       [0.61843982, 1.17557824, 1.20756543, ..., 1.09996001, 1.28553055,
        0.41613339],
       [0.39093072, 0.75116381, 0.74169761, ..., 0.6802466 , 0.79080765,
        0.21991998],
       [0.62404415, 1.15449302, 1.30377619, ..., 1.16930443, 1.38313206,
        0.58989106]])

In [ ]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [ ]:
comparison_data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, 0.6409307354550441)","(8.79, 1.2166262172026236)","(99.0, 1.2560604579985322)","(-8.16, 0.5701568757704916)","(-7.52, 0.6137639919961214)","(-8.5, 0.6734382388352531)","(-9.85, 0.537444832535184)","(99.0, 0.6290951061579736)","(-8.98, 1.0660841469756734)","(-4.76, 0.5330042572765548)","(-8.5, 0.9834177668440448)","(-6.75, 0.8406175023664819)","(99.0, 0.5642457182612401)","(99.0, 1.0590816134470549)","(99.0, 0.9008911848553919)","(-7.52, 0.3880607194799908)","(-7.43, 0.9182975040694336)","(99.0, 0.464251168013899)","(-9.85, 0.6483777080384575)","(-9.85, 1.2426808359112365)","(-9.37, 0.6304656602086041)","(1.5, 0.42784392799005544)","(-4.37, 0.4758376233079857)","(-9.81, 0.6867579463906753)","(-8.5, 0.3035079131355734)","(1.12, 0.19700478240379204)","(7.82, 0.587849387826321)","(2.86, 0.8995270641456086)","(9.13, 1.2675803636987237)","(-7.43, 0.2873081788932786)","(2.14, 1.2506674497388857)","(-4.08, 0.3442598062950012)","(-9.08, 1.1722827968700582)","(7.82, 1.1227012295509384)","(5.05, 1.1599492337766841)","(4.95, 0.5570928133683026)","(-9.17, 0.761318147783942)","(-8.4, 0.367414143505538)","(-8.4, 0.27612560458622015)","(-8.4, 0.6165296981758916)",...,"(8.59, 0.05997782686239974)","(3.59, 0.5420844581382946)","(-6.84, 0.9589196167554178)","(-9.03, 1.3166733926742176)","(2.82, 0.90252180654312)","(-1.36, 0.37624447132771893)","(-9.08, 0.6698510907281202)","(8.3, 0.40763595535818453)","(5.68, 0.9379708155356855)","(-4.81, 1.2088640793362744)","(nan, 0.9661194337545862)","(nan, 0.4805587719854066)","(nan, 0.8911748370056042)","(nan, 0.4894852028663555)","(nan, 1.0105624108958084)","(nan, 0.824159352411743)","(nan, 0.9628338960859708)","(-9.42, 0.42396001411862005)","(nan, 1.0173811894721414)","(nan, 1.2048724026383957)","(nan, 0.2972016085868658)","(-7.72, 0.8062684549372505)","(nan, 1.0882550157378317)","(nan, 0.6933875884333492)","(nan, 0.8531227226325284)","(nan, 0.16240547358264137)","(nan, 0.5784847249760606)","(nan, 0.9016072497378373)","(nan, 1.1046679466335563)","(nan, 0.6632096893536242)","(2.82, 1.1611996094954056)","(nan, 1.0928579845870399)","(nan, 0.2740336943681056)","(nan, 0.5184728999831163)","(nan, 0.8233216768957448)","(nan, 0.8861988176392022)","(-5.63, 0.8158815603395883)","(nan, 1.1431512601297356)","(nan, 1.3368976985769592)","(nan, 0.4403960599458966)"
1,"(4.08, 0.18876859056578627)","(-0.29, 0.32254424059182707)","(6.36, 0.46606328055339336)","(4.37, 0.3376437392072473)","(-2.38, 0.10583174180752047)","(-9.66, 0.051967407548870685)","(-0.73, 0.1571791262279014)","(-5.34, 0.44850360372886516)","(8.88, 0.3614585107943023)","(9.22, 0.1987041933533052)","(6.75, 0.1261082618337238)","(8.64, 0.30766426554179926)","(4.42, 0.1061575821315548)","(7.43, 0.45118138384701817)","(4.56, 0.05077787008942738)","(-0.97, 0.1336362927832543)","(4.66, 0.25016144190043693)","(-0.68, 0.17290903568525673)","(3.3, 0.1822401740173918)","(-1.21, 0.4345769675120081)","(0.87, 0.06955571616629239)","(8.64, 0.03612543638185019)","(99.0, 0.14722675959698114)","(9.17, 0.15386907604083505)","(0.05, 0.07112303347904543)","(7.57, 0.07608365849120599)","(4.71, 0.22381580054071548)","(0.87, 0.384479217048814)","(-0.39, 0.4338681765692535)","(6.99, 0.01428107070493716)","(6.5, 0.37099637124707113)","(-0.92, 0.15974951072814053)","(7.14, 0.36608887456324285)","(9.03, 0.30342144241171315)","(-1.8, 0.491744321931209)","(0.73, 0.1391381417347746)","(7.09, 0.2708846600070514)","(3.4, 0.11842068625400624)","(-0.87, 0.04363647945509049)","(7.91, 0.059942518305205895)",...,"(-6.7, 0.007311966195535436)","(-3.35, 0.12232103810513344)","(-9.03, 0.12842382862924437)","(4.47, 0.4280729022805887)","(4.08, 0.40976180214473074)","(-3.83, 0.25569202712256395)","(8.74, 0.05296981875322723)","(1.12, 0.1520882226515040

In [ ]:
#Changing the number of latent factors while all the other hyper-parameters stay the same
n_latent_factors2=4
user_ratings2= new_arr
latent_user_preferences2=np.random.random((user_ratings2.shape[0], n_latent_factors2))
latent_item_features2=np.random.random((user_ratings2.shape[1],n_latent_factors2))

predictions2 = latent_user_preferences2.dot(latent_item_features2.T)
values2 = [zip(user_ratings2[i], predictions2[i]) for i in range(predictions2.shape[0])]
comparison_data2 = pd.DataFrame(values2)
comparison_data2.columns = df.columns

In [ ]:
comparison_data2

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, 1.1927514545651399)","(8.79, 1.418528382350598)","(99.0, 0.7906535703684382)","(-8.16, 0.18338420607699862)","(-7.52, 0.30975840452548753)","(-8.5, 1.1041981422104767)","(-9.85, 0.45784440388245007)","(99.0, 0.7846569226234615)","(-8.98, 1.2428860296152757)","(-4.76, 0.6810146436018026)","(-8.5, 0.9303863447617926)","(-6.75, 0.7166575303209118)","(99.0, 0.6861192511899317)","(99.0, 0.8599794836258414)","(99.0, 0.9183538230943343)","(-7.52, 1.100449469571866)","(-7.43, 0.6003611637586889)","(99.0, 0.9342349206951037)","(-9.85, 0.8272654138668434)","(-9.85, 0.8863895835005373)","(-9.37, 1.0606414099938242)","(1.5, 0.4888627978368056)","(-4.37, 0.871822135380533)","(-9.81, 1.007872518727604)","(-8.5, 1.0767545438467334)","(1.12, 0.6255023170456204)","(7.82, 0.9152123862239054)","(2.86, 1.0760645403843043)","(9.13, 0.5629435228490716)","(-7.43, 0.6048491234919023)","(2.14, 0.6716978762663399)","(-4.08, 0.5222951425104404)","(-9.08, 0.7878082804235215)","(7.82, 0.9072264201593555)","(5.05, 0.6147600819287655)","(4.95, 0.889944968484441)","(-9.17, 0.9051240955065161)","(-8.4, 0.7714353074211435)","(-8.4, 1.0084145036347214)","(-8.4, 0.9729207355996228)",...,"(8.59, 1.1824966535612913)","(3.59, 0.7801448215648502)","(-6.84, 0.9772540766652026)","(-9.03, 0.6183297200613241)","(2.82, 0.3234385354558173)","(-1.36, 0.3913111606616456)","(-9.08, 0.9434560343494658)","(8.3, 0.874543519252783)","(5.68, 0.7692499950092359)","(-4.81, 1.0739601385116684)","(nan, 0.5586669320064043)","(nan, 0.965867517403227)","(nan, 0.8147106340539849)","(nan, 1.138773438233862)","(nan, 0.34513104165424413)","(nan, 0.4406474492999986)","(nan, 0.8148313990017214)","(-9.42, 0.3843069254937423)","(nan, 0.7268962504160292)","(nan, 0.5536570632419945)","(nan, 0.38947303054815696)","(-7.72, 1.0822045310403876)","(nan, 0.6468849160668176)","(nan, 1.15747991605873)","(nan, 1.1568684516648489)","(nan, 0.880264242897396)","(nan, 0.42506446167967954)","(nan, 1.2481409695155574)","(nan, 0.1705841729594792)","(nan, 0.9632937910696607)","(2.82, 1.173950513270387)","(nan, 0.7828662400705513)","(nan, 1.2368257778519445)","(nan, 0.9810509524694642)","(nan, 0.4928881734983906)","(nan, 1.1274687344359964)","(-5.63, 0.16104109696125915)","(nan, 1.416180663150299)","(nan, 0.5794372305035085)","(nan, 0.8302802300345409)"
1,"(4.08, 1.2072788275281536)","(-0.29, 1.657598073455405)","(6.36, 1.0658048285472386)","(4.37, 0.24863634684355362)","(-2.38, 0.503236185402368)","(-9.66, 1.3621562848933677)","(-0.73, 0.5569304191298216)","(-5.34, 1.1011537493161203)","(8.88, 1.2779502588684308)","(9.22, 1.0286851878007919)","(6.75, 1.322711708910822)","(8.64, 0.8657423183395638)","(4.42, 0.7764915211038455)","(7.43, 1.1717189525205993)","(4.56, 0.9059640478545087)","(-0.97, 1.4305929800758805)","(4.66, 0.8992680868637688)","(-0.68, 1.202098848235512)","(3.3, 0.9135837323833318)","(-1.21, 1.2376900582966635)","(0.87, 1.3048326783374238)","(8.64, 0.6141641281523086)","(99.0, 1.2929326555115668)","(9.17, 1.4386817484960466)","(0.05, 1.2602904897094258)","(7.57, 0.9476225596260228)","(4.71, 1.1149821179228052)","(0.87, 1.4520192064820894)","(-0.39, 0.937533781973859)","(6.99, 0.7178276656387897)","(6.5, 1.1360528791218238)","(-0.92, 0.8973485820931234)","(7.14, 1.217847756836554)","(9.03, 1.2090646419338875)","(-1.8, 0.6689462504298294)","(0.73, 1.0857925392635441)","(7.09, 0.9562098624204598)","(3.4, 1.238705130413742)","(-0.87, 1.004422064956357)","(7.91, 1.4672847677457286)",...,"(-6.7, 1.4628688286115288)","(-3.35, 0.9997246800537574)","(-9.03, 1.050489676942771)","(4.47, 0.9742654529622238)","(4.08, 0.5533117085623381)","(-3.83, 0.6696516842596578)","(8.74, 1.0586321001977197)","(1.12, 1.3554343438707337)","(0.78, 1.1963364611492409)","(7.52, 1.39619